## ICESat-2 ATL03 SlideRule Download

This code is adapted from Sliderule Python example code. It is designed to pull ALT03 points for a region of interest and save those elevations as a CSV. 

Activate the sliderule_env enviroment to run

Last updated by Karina Zikan July 2023

In [1]:
import sys
import logging
import concurrent.futures
import time
from datetime import datetime
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS
from shapely.geometry import Polygon, Point
import sliderule
from sliderule import sliderule, icesat2, earthdata, h5

## SlideRule Configuration

In [2]:
# Configure ICESat-2 API
icesat2.init("slideruleearth.io", verbose=False)

## Specify Region of Interest

#### 1. Make geojson from shapefile - if you have a geojson of the region skip to 2.

In [ ]:
# Specify region of interest shapefile
shp = "Sites/RCEW/ROIs/RCEW-outline_WGS84.shp"
shapefile = gpd.read_file(shp)
# shapefile.to_file('DryCreek/boundDCEW/boundDCEW_WGS84.geojson', driver='GeoJSON')
shapefile.crs is None

##### - Check shapefile crs

In [ ]:
## if shapefile.crs is None is false check the crs
shapefile.crs

##### - Make Geojson

In [ ]:
# shapefile.to_file('Sites/RCEW/ROIs/RCEW-outline_WGS84.geojson', driver='GeoJSON')

In [ ]:
## if shapefile.crs is None is true run this section
#crs = 'epsg:8826' # set the starting datum using the epsg code
#icesat_crs = 4979 # epsg code for WGS84
#shapefile = shapefile.set_crs(crs)
#shapefile = shapefile.to_crs(icesat_crs) # transform to WGS84 
#shapefile.to_file('/Mores/MCSboundary/MoresBound_WGS84.geojson', driver='GeoJSON')

In [ ]:
## if shapefile.crs is not WGS84 run this section
#icesat_crs = 4979 # epsg code for WGS84
#shapefile = shapefile.to_crs(icesat_crs) # transform to WGS84 
#shapefile.to_file('/Mores/MCSboundary/MoresBound_WGS84.geojson', driver='GeoJSON')

#### 2. Read in geojson for Region of Interest

In [ ]:
# Specify region of interest from geojson
poly_fn = 'Sites/RCEW/ROIs/RCEW-outline_WGS84.geojson'
region = sliderule.toregion(poly_fn)["poly"] # symplify 
region

In [ ]:
# Prepare coordinate lists for plotting the region of interest polygon
region_lon = [e["lon"] for e in region]
region_lat = [e["lat"] for e in region]

In [ ]:
# Read geojson with geopandas
pregion = gpd.read_file(poly_fn)
map = pregion.plot(); # plot origonal boundery 
map.plot(region_lon, region_lat, linewidth=1, color='g'); # plot symplified region boundery for a sanity check

#### 3. Save simplified polygon

In [ ]:
polygon_geom = Polygon(zip(region_lon, region_lat))
crs = 'epsg:4979'
polygon = gpd.GeoDataFrame(index=[0], geometry=[polygon_geom])       

polygon

### Save to file if you haven't already

In [ ]:
polygon.to_file('Sites/RCEW/ROIs/RCEW-outline_WGS84_simple.geojson', driver='GeoJSON')

# Sliderule Product processing

Follow the processing section for the desired product

### ATL03 processing

#### ATL03 Parameters

In [ ]:
parms = {
    "poly": region,
    "srt": 0,
    "atl08_class": "atl08_ground",
    "cnf": icesat2.CNF_BACKGROUND, 
    "ats": 20.0,
    "cnt": 10, 
    "len": 40.0,
    "res": 20.0,
    "maxi": 6
}
cvs_name = "Sites/RCEW/IS2_Data/RCEW-ICESat2-ATL03.csv" #set name of the output

#### Calculate Elevations

In [ ]:
# Latch Start Time
perf_start = time.perf_counter()

# Request ATL06 Data
data = icesat2.atl03sp(parms)

# Latch Stop Time
perf_stop = time.perf_counter()

# Display Statistics
perf_duration = perf_stop - perf_start
print("Completed in {:.3f} seconds of wall-clock time".format(perf_duration))
#print("Reference Ground Tracks: {}".format(atl06_sr["rgt"].unique()))
#print("Cycles: {}".format(atl06_sr["cycle"].unique()))
print("Received {} elevations".format(data.shape[0]))

## Data Check

In [ ]:
#Set utmzone code - utm zone 11:32611
utm_code = 'epsg:32611' 

data['Longitude'] = data['geometry'].x
data['Latitude'] = data['geometry'].y
eastnorths = data['geometry'].to_crs(utm_code)
data['Easting'] = eastnorths.x
data['Northing'] = eastnorths.y
data

## Write CSV - ATL03

#### filter to summer months

In [ ]:
import datetime 

In [ ]:
data['Date'] = pd.to_datetime(data.index)
data = data[(data['Date'].dt.month >= 5) & (data['Date'].dt.month <= 9)]
data[1:6]

In [ ]:
data_xyz = pd.DataFrame()

data_xyz['X'] = data['Easting']
data_xyz['Y'] = data['Northing']
data_xyz['Z'] = data['height']
data_xyz['rgt'] = data['rgt']
data_xyz['time'] = data['Date']

data_xyz = data_xyz.set_index('time')
data_xyz

#### Write CSV

In [ ]:
data_xyz.to_csv(cvs_name)